### 3.2 Query Optimization

In [0]:
%run ./Includes/Classroom-Setup

In [0]:
df = spark.read.parquet(eventsPath)
display(df)

device,ecommerce,event_name,event_previous_timestamp,event_timestamp,geo,items,traffic_source,user_first_touch_timestamp,user_id
macOS,"List(null, null, null)",warranty,1593878899217692,1593878946592107,"List(Montrose, MI)",List(),google,1593878899217692,UA000000107379500
Windows,"List(null, null, null)",press,1593876662175340,1593877011756535,"List(Northampton, MA)",List(),google,1593876662175340,UA000000107359357
macOS,"List(null, null, null)",add_item,1593878792892652,1593878815459100,"List(Salinas, CA)","List(List(null, M_STAN_T, Standard Twin Mattress, 595.0, 595.0, 1))",youtube,1593878455472030,UA000000107375547
iOS,"List(null, null, null)",mattresses,1593878178791663,1593878809276923,"List(Everett, MA)",List(),facebook,1593877903116176,UA000000107370581
Windows,"List(null, null, null)",mattresses,null,1593878628143633,"List(Cottage Grove, MN)",List(),google,1593878628143633,UA000000107377108
Windows,"List(null, null, null)",main,null,1593878634344194,"List(Medina, MN)",List(),youtube,1593878634344194,UA000000107377161
iOS,"List(null, null, null)",main,null,1593877936171803,"List(Mount Pleasant, UT)",List(),direct,1593877936171803,UA000000107370851
macOS,"List(null, null, null)",main,null,1593876843215329,"List(Piedmont, AL)",List(),instagram,1593876843215329,UA000000107360961
Android,"List(null, null, null)",warranty,1593878529774474,1593879213196400,"List(Rancho Santa Margarita, CA)",List(),instagram,1593878529774474,UA000000107376205
Windows,"List(null, null, null)",main,null,1593876713246514,"List(Elyria, OH)",List(),facebook,1593876713246514,UA000000107359805


* Logical Optimization

In [0]:
from pyspark.sql.functions import col

limitEventsDF = (df
  .filter(col("event_name") != "reviews")
  .filter(col("event_name") != "checkout")
  .filter(col("event_name") != "register")
  .filter(col("event_name") != "email_coupon")
  .filter(col("event_name") != "cc_info")
  .filter(col("event_name") != "delivery")
  .filter(col("event_name") != "shipping_info")
  .filter(col("event_name") != "press")
)

limitEventsDF.count()

Out[5]: 7691469

`explain()`

In [0]:
limitEventsDF.explain(True)

== Parsed Logical Plan ==
'Filter NOT ('event_name = press)
+- Filter NOT (event_name#4673 = shipping_info)
   +- Filter NOT (event_name#4673 = delivery)
      +- Filter NOT (event_name#4673 = cc_info)
         +- Filter NOT (event_name#4673 = email_coupon)
            +- Filter NOT (event_name#4673 = register)
               +- Filter NOT (event_name#4673 = checkout)
                  +- Filter NOT (event_name#4673 = reviews)
                     +- Relation[device#4671,ecommerce#4672,event_name#4673,event_previous_timestamp#4674L,event_timestamp#4675L,geo#4676,items#4677,traffic_source#4678,user_first_touch_timestamp#4679L,user_id#4680] parquet

== Analyzed Logical Plan ==
device: string, ecommerce: struct<purchase_revenue_in_usd:double,total_item_quantity:bigint,unique_items:bigint>, event_name: string, event_previous_timestamp: bigint, event_timestamp: bigint, geo: struct<city:string,state:string>, items: array<struct<coupon:string,item_id:string,item_name:string,item_revenue_in_us

* betterDF

In [0]:
betterDF = (df.filter(
  (col("event_name").isNotNull()) &
  (col("event_name") != "reviews") &
  (col("event_name") != "checkout") &
  (col("event_name") != "register") &
  (col("event_name") != "email_coupon") &
  (col("event_name") != "cc_info") &
  (col("event_name") != "delivery") &
  (col("event_name") != "shipping_info") &
  (col("event_name") != "press")
))

betterDF.count()

betterDF.explain(True)

== Parsed Logical Plan ==
'Filter ((((((((isnotnull('event_name) AND NOT ('event_name = reviews)) AND NOT ('event_name = checkout)) AND NOT ('event_name = register)) AND NOT ('event_name = email_coupon)) AND NOT ('event_name = cc_info)) AND NOT ('event_name = delivery)) AND NOT ('event_name = shipping_info)) AND NOT ('event_name = press))
+- Relation[device#4671,ecommerce#4672,event_name#4673,event_previous_timestamp#4674L,event_timestamp#4675L,geo#4676,items#4677,traffic_source#4678,user_first_touch_timestamp#4679L,user_id#4680] parquet

== Analyzed Logical Plan ==
device: string, ecommerce: struct<purchase_revenue_in_usd:double,total_item_quantity:bigint,unique_items:bigint>, event_name: string, event_previous_timestamp: bigint, event_timestamp: bigint, geo: struct<city:string,state:string>, items: array<struct<coupon:string,item_id:string,item_name:string,item_revenue_in_usd:double,price_in_usd:double,quantity:bigint>>, traffic_source: string, user_first_touch_timestamp: bigint, use

* stupidDF

In [0]:
stupidDF = (df
  .filter(col("event_name") != "finalize")
  .filter(col("event_name") != "finalize")
  .filter(col("event_name") != "finalize")
  .filter(col("event_name") != "finalize")
  .filter(col("event_name") != "finalize")
)

stupidDF.explain(True)

== Parsed Logical Plan ==
'Filter NOT ('event_name = finalize)
+- Filter NOT (event_name#4673 = finalize)
   +- Filter NOT (event_name#4673 = finalize)
      +- Filter NOT (event_name#4673 = finalize)
         +- Filter NOT (event_name#4673 = finalize)
            +- Relation[device#4671,ecommerce#4672,event_name#4673,event_previous_timestamp#4674L,event_timestamp#4675L,geo#4676,items#4677,traffic_source#4678,user_first_touch_timestamp#4679L,user_id#4680] parquet

== Analyzed Logical Plan ==
device: string, ecommerce: struct<purchase_revenue_in_usd:double,total_item_quantity:bigint,unique_items:bigint>, event_name: string, event_previous_timestamp: bigint, event_timestamp: bigint, geo: struct<city:string,state:string>, items: array<struct<coupon:string,item_id:string,item_name:string,item_revenue_in_usd:double,price_in_usd:double,quantity:bigint>>, traffic_source: string, user_first_touch_timestamp: bigint, user_id: string
Filter NOT (event_name#4673 = finalize)
+- Filter NOT (event_na

### 3.3 Partitioning

`rdd.getNumPartitions()`

In [0]:
df = spark.read.parquet(eventsPath)
df.rdd.getNumPartitions()

Out[9]: 4

In [0]:
# Access SparkContext through SparkSession to get the number of cores or slots

print(spark.sparkContext.defaultParallelism)
# print(sc.defaultParallelism)

8


`repartition()`

In [0]:
repartitionedDF = df.repartition(7)
repartitionedDF.rdd.getNumPartitions()

Out[11]: 7

`coalesce()`

In [0]:
# It will stay at the smallest number of partitions if a larger number of partitions is requested
coalesceDF = df.coalesce(10)
coalesceDF.rdd.getNumPartitions()

Out[12]: 4

`SparkConf()`

In [0]:
spark.conf.get("spark.sql.shuffle.partitions")

Out[13]: '200'

In [0]:
spark.conf.set("spark.sql.shuffle.partitions", "8")

`Adaptive Query Execution`

In [0]:
spark.conf.get("spark.sql.adaptive.enabled")

Out[15]: 'true'

### 3.4 Review

In [0]:
sourceFile = "dbfs:/mnt/training/dataframes/people-with-dups.txt"
destFile = workingDir + "/people.parquet"
deltaDestFile = workingDir + "/people"

dbutils.fs.rm(destFile, True)

Out[16]: False

In [0]:
df = spark.read.option("header", True).option("inferSchema", True).option("sep", ":").csv(sourceFile)
display(df)

firstName,middleName,lastName,gender,birthDate,salary,ssn
Emanuel,Wallace,Panton,M,1988-03-04,101255,935-90-7627
Eloisa,Rubye,Cayouette,F,2000-06-20,204031,935-89-9009
Cathi,Svetlana,Prins,F,2012-12-22,35895,959-30-7957
Mitchel,Andres,Mozdzierz,M,1966-05-06,55108,989-27-8093
Angla,Melba,Hartzheim,F,1938-07-26,13199,935-27-4276
Rachel,Marlin,Borremans,F,1923-02-23,67070,996-41-8616
Catarina,Phylicia,Dominic,F,1969-09-29,201021,999-84-8888
Antione,Randy,Hamacher,M,2004-03-05,271486,917-96-3554
Madaline,Shawanda,Piszczek,F,1996-03-17,183944,963-87-9974
Luciano,Norbert,Sarcone,M,1962-12-14,73069,909-96-1669


`initcap(), concat_ws()`

In [0]:
from pyspark.sql.functions import *

df_tmp = (
  df.withColumn("firstName", initcap(col("firstName"))).\
  withColumn("middleName", initcap("middleName")).\
  withColumn("lastName", initcap("lastName")).\
  withColumn("ssn", split(col("ssn"), "-", 3)).\
  withColumn("ssn", concat_ws("", "ssn")).\
  dropDuplicates()
)

display(df_tmp)

firstName,middleName,lastName,gender,birthDate,salary,ssn
Angla,Melba,Hartzheim,F,1938-07-26,13199,935274276
Madaline,Shawanda,Piszczek,F,1996-03-17,183944,963879974
Jenni,Janella,Mcquiston,F,1969-07-11,137418,932557164
Shantelle,Zoe,Spicher,F,1952-12-11,183419,989798274
Deana,Marcie,Malle,F,2011-06-24,15388,973529173
Claudio,Leopoldo,Ealand,M,1916-01-14,158319,929616825
Andrew,Darnell,Kniceley,M,1966-09-02,25955,987264880
Jonelle,Maryam,Berkstresser,F,1944-11-17,253619,935627974
Lynsey,Marcelina,Dagrella,F,1969-03-01,209989,978427296
Max,Fletcher,Hoople,M,1996-04-08,205848,953506974


`repartition()`

In [0]:
(
  df_tmp.repartition(numPartitions=8).\
  write.\
  mode("overwrite").\
  option("compression", "snappy").\
  parquet(destFile)
)

final_df = spark.read.parquet(destFile)
display(final_df)

firstName,middleName,lastName,gender,birthDate,salary,ssn
Barb,Tequila,Ehle,F,2002-07-14,125647,900903192
Brendan,Pedro,Grills,M,1996-06-22,102040,974184852
Sung,Walker,Liebold,M,1924-03-22,25348,961221263
Casey,Dewey,Phagan,M,1988-06-27,72271,911299683
Reginald,Jimmie,Leppla,M,1919-12-22,17567,932969852
Giuseppe,Burton,Fitzgerlad,M,1973-06-27,74842,963566848
Sharonda,Else,Lindquist,F,1958-06-01,283164,938656777
Dustin,Franklin,Purviance,M,1923-05-12,134560,966735266
Cassi,Valeria,Giusto,F,1976-03-09,218505,942538238
Alyssa,Sharolyn,Singco,F,1929-04-07,53233,944501303


In [0]:
(
  df_tmp.write.mode("overwrite").\
  format("delta").\
  mode("overwrite").save(deltaDestFile)
)

final_df = spark.read.format("delta").load(deltaDestFile)
display(final_df)

firstName,middleName,lastName,gender,birthDate,salary,ssn
Angla,Melba,Hartzheim,F,1938-07-26,13199,935274276
Madaline,Shawanda,Piszczek,F,1996-03-17,183944,963879974
Jenni,Janella,Mcquiston,F,1969-07-11,137418,932557164
Shantelle,Zoe,Spicher,F,1952-12-11,183419,989798274
Deana,Marcie,Malle,F,2011-06-24,15388,973529173
Claudio,Leopoldo,Ealand,M,1916-01-14,158319,929616825
Andrew,Darnell,Kniceley,M,1966-09-02,25955,987264880
Jonelle,Maryam,Berkstresser,F,1944-11-17,253619,935627974
Lynsey,Marcelina,Dagrella,F,1969-03-01,209989,978427296
Max,Fletcher,Hoople,M,1996-04-08,205848,953506974


In [0]:
%run "./Includes/Classroom-Cleanup"

Dropped database and removed files in working directory

Datasets mounted and student environment set up